# Test with 3-D input

In [34]:
import sys, os
# get the parent directory of this notebook
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from learning_loop.active_learning import active_learning_loop
# 
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize_scalar
from scipy.optimize import differential_evolution

In [37]:
def f(x):
    """
    3D nonlinear function with a global minimum near (0.3, 0.6, 0.9).
    x is expected to be an array-like of shape (3,)
    """
    x = np.asarray(x)
    return (x[0] - 0.3)**2 + (x[1] - 0.6)**2 + (x[2] - 0.9)**2 \
           + 0.5 * np.sin(5 * x[0]) * np.cos(3 * x[1]) + 0.4
X = np.random.rand(1000000, 3)
Y = np.array([f(x) for x in X])
idx = np.argmin(Y)
print("Brute Force with random search:", X[idx], Y[idx])
#
bounds = [(0,1), (0,1), (0,1)]  # 3-D box
result = differential_evolution(f, bounds, seed=42, polish=True)
print("Global minimum via differential_evolution:")
print(f"  x* = {result.x}")
print(f"  f(x*) = {result.fun:.6f}")

Brute Force with random search: [0.31309502 0.9044011  0.8999511 ] 0.038019949972052436
Global minimum via differential_evolution:
  x* = [0.31204703 0.90674042 0.90000007]
  f(x*) = 0.037996


In [38]:
X0 = np.array([[0.1, 0.1, 0.1]])        # initial single observation
Y0 = np.array([f(X0[0])])               # corresponding function value
# Candidate pool:  1000 random samples in [0,1]^3
X_grid = np.random.rand(1000, 3)
# Neural network configuration
nn_cfg = {
    "hidden_layers": [64, 32, 16],
    "activation": "relu",     # try "tanh" for smoother landscapes
    "dropout": 0.1,
    "epochs": 300,
    "lr": 1e-3,
}
# active learning loop
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=200,
    M=8,                      # ensemble size
    model_type="nn",
    nn_config=nn_cfg,
    bootstrap=False,            # helps uncertainty early on
    acquisition="LCB",          # try "LCB" too for comparison
    patience=50,
    min_improve_pct=0.001,      # run until all candidates evaluated
)

# retrieve data for plotting
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])   # Y values evaluated each iteration
best_y = np.array([h[3] for h in history])   # running best values
best_final = np.min(best_y)
#
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters + 1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=result.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {result.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Neural Network)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white",
    width=800,
    height=500
)
fig.show()


Starting active learning run — initial best_y = 1.5590
[001/200] acq=LCB, x_next=[0.9213, 0.0345, 0.0387], y_next=1.3531 (new), best_y=1.3531
[002/200] acq=LCB, x_next=[0.943 , 0.0169, 0.1049], y_next=1.2864 (new), best_y=1.2864
[003/200] acq=LCB, x_next=[0.665 , 0.0247, 0.0471], y_next=1.5007 (new), best_y=1.2864
[004/200] acq=LCB, x_next=[9.7886e-01, 5.7604e-04, 2.3635e-01], y_next=1.1688 (new), best_y=1.1688
[005/200] acq=LCB, x_next=[0.9968, 0.0136, 0.7831], y_next=0.7619 (new), best_y=0.7619
[006/200] acq=LCB, x_next=[0.8991, 0.0502, 0.9537], y_next=0.5813 (new), best_y=0.5813
[007/200] acq=LCB, x_next=[0.6577, 0.0213, 0.9524], y_next=0.7926 (new), best_y=0.5813
[008/200] acq=LCB, x_next=[0.8721, 0.0682, 0.9055], y_next=0.5506 (new), best_y=0.5506
[009/200] acq=LCB, x_next=[0.8458, 0.228 , 0.9215], y_next=0.4936 (new), best_y=0.4936
[010/200] acq=LCB, x_next=[0.967 , 0.8047, 0.9966], y_next=1.2668 (new), best_y=0.4936
[011/200] acq=LCB, x_next=[0.7786, 0.1214, 0.9085], y_next=0.53

In [30]:
X0 = np.array([[0.1, 0.1, 0.1]])        # initial single observation
Y0 = np.array([f(X0[0])])               # corresponding function value
# Candidate pool:  1000 random samples in [0,1]^3
X_grid = np.random.rand(1000, 3)
# active learning loop
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=200,
    M=8,                      # ensemble size
    model_type="poly",
    bootstrap=False,            # helps uncertainty early on
    acquisition="LCB",          # try "LCB" too for comparison
    patience=50,
    min_improve_pct=0.001,      # run until all candidates evaluated
)

# retrieve data for plotting
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])   # Y values evaluated each iteration
best_y = np.array([h[3] for h in history])   # running best values
best_final = np.min(best_y)
#
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters + 1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=result.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {result.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Polynomial)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white",
    width=800,
    height=500
)
fig.show()


Starting active learning run — initial best_y = 1.1590
[001/200] acq=LCB, x_next=[0.8925, 0.2842, 0.0174], y_next=0.9110 (new), best_y=0.9110
[002/200] acq=LCB, x_next=[0.9902, 0.8944, 0.0503], y_next=1.7207 (new), best_y=0.9110
[003/200] acq=LCB, x_next=[0.9912, 0.0063, 0.3705], y_next=0.6254 (new), best_y=0.6254
[004/200] acq=LCB, x_next=[0.9645, 0.0205, 0.7924], y_next=0.2930 (new), best_y=0.2930
[005/200] acq=LCB, x_next=[0.9435, 0.1009, 0.9513], y_next=0.1885 (new), best_y=0.1885
[006/200] acq=LCB, x_next=[0.9035, 0.0678, 0.9485], y_next=0.1694 (new), best_y=0.1694
[007/200] acq=LCB, x_next=[0.7326, 0.0201, 0.9903], y_next=0.2830 (new), best_y=0.1694
[008/200] acq=LCB, x_next=[0.907 , 0.0615, 0.9099], y_next=0.1747 (new), best_y=0.1694
[009/200] acq=LCB, x_next=[0.8234, 0.8026, 0.9815], y_next=0.6291 (new), best_y=0.1694
[010/200] acq=LCB, x_next=[0.8121, 0.2116, 0.8618], y_next=0.0945 (new), best_y=0.0945
[011/200] acq=LCB, x_next=[0.7834, 0.3009, 0.9635], y_next=0.1103 (new), be

In [42]:
X0 = np.array([[0.1, 0.1, 0.1]])        # initial single observation
Y0 = np.array([f(X0[0])])               # corresponding function value
# Candidate pool:  1000 random samples in [0,1]^3
X_grid = np.random.rand(1000, 3)
# active learning loop
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=200,
    M=8,                      # ensemble size
    model_type="tree",
    bootstrap=False,            # helps uncertainty early on
    acquisition="LCB",          # try "LCB" too for comparison
    patience=50,
    min_improve_pct=0.001,      # run until all candidates evaluated
)

# retrieve data for plotting
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])   # Y values evaluated each iteration
best_y = np.array([h[3] for h in history])   # running best values
best_final = np.min(best_y)
#
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters + 1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=result.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {result.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Random Forest)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white",
    width=800,
    height=500
)
fig.show()


Starting active learning run — initial best_y = 1.5590
[001/200] acq=LCB, x_next=[0.6324, 0.4898, 0.1618], y_next=1.0665 (new), best_y=1.0665
[002/200] acq=LCB, x_next=[0.7084, 0.542 , 0.1082], y_next=1.2078 (new), best_y=1.0665
[003/200] acq=LCB, x_next=[0.6558, 0.1829, 0.7519], y_next=0.6640 (new), best_y=0.6640
[004/200] acq=LCB, x_next=[0.2742, 0.1821, 0.5357], y_next=1.1267 (new), best_y=0.6640
[005/200] acq=LCB, x_next=[0.6305, 0.9778, 0.9832], y_next=0.6642 (new), best_y=0.6640
[006/200] acq=LCB, x_next=[0.1036, 0.1599, 0.8601], y_next=0.8534 (new), best_y=0.6640
[007/200] acq=LCB, x_next=[0.156 , 0.2393, 0.756 ], y_next=0.8364 (new), best_y=0.6640
[008/200] acq=LCB, x_next=[0.9349, 0.6489, 0.8335], y_next=0.9934 (new), best_y=0.6640
[009/200] acq=LCB, x_next=[0.534 , 0.2988, 0.9507], y_next=0.6899 (new), best_y=0.6640
[010/200] acq=LCB, x_next=[0.6256, 0.3252, 0.8713], y_next=0.5862 (new), best_y=0.5862
[011/200] acq=LCB, x_next=[0.5571, 0.0699, 0.8504], y_next=0.9201 (new), be